Register Rabi's miRNA stuff


In [1]:
import pandas
import gcat

In [2]:
from curation_common import *

In [3]:
from htsworkflow.submission.encoded import typed_column_parser

In [4]:
encode = ENCODED('www.encodeproject.org')
encode.load_netrc()

In [5]:
def validate_aliases(sheet, accession_name='accession'):
    # need to add 2 to python index to match 
    # whats the spreadsheet row number.
    # one for zero based vs one based, and one for the
    # header row.
    fixes = []
    lo_row_offset = 2
    for i, row in sheet.iterrows():
        alias = row['aliases:array']
        accession = row[accession_name]
        if ' ' in alias:
            fixed_alias =  alias.replace(' ', '-')
            fixes.append([i + lo_row_offset, accession, alias, fixed_alias])
    return pandas.DataFrame(fixes, columns=['row number', accession_name, 'old alias', 'aliases:array'])

# create biosamples

In [6]:
#biosamples = pandas.read_excel('ENCODE_miRNA_experiments_02042016.xlsx', sheetname='Biosamples')
spreadsheet_name = 'ENCODE_miRNA_experiments_02042016'
rabi = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosamples = rabi.parse('Biosamples', header=0)


In [7]:
validate_aliases(biosamples)

,row number,accession,old alias,aliases:array


In [8]:
created = []
if not biosamples.empty:
    created = encode.post_sheet('/biosamples/', biosamples, verbose=True, dry_run=True)
    print(len(created))

In [9]:
if created:
    updates = []
    for row in created:
        updates.append({'uuid': row['uuid'], 'biosample_accession': row['accession'], 'aliases': row['aliases'][0]})
    updatesdf = pandas.DataFrame(updates, columns=['uuid', 'biosample_accession', 'aliases'])
    updatesdf.to_excel('/tmp/biosamples.xlsx')

# Create Experiments

In [10]:
#experiments = pandas.read_excel('ENCODE_miRNA_experiments_02042016.xlsx', sheetname='Experiments')
rabi = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = rabi.parse('Experiments', header=0)

In [11]:
#experiment_alias_fixes = validate_aliases(experiments, 'experiment_accession')
#experiment_alias_fixes.to_csv('/tmp/experiments.csv')
#experiment_alias_fixes

In [12]:
created = []
if not experiments.empty:
    created = encode.post_sheet('/experiments/', experiments, dry_run=True, verbose=True)
    print(len(created))
    

0


In [13]:
if created:
    updatesdf = pandas.DataFrame(created, columns=['uuid', 'accession','description'])
    updatesdf.to_excel('/tmp/experiments.xlsx', index=False)

# Create Libraries

In [14]:
#libraries = pandas.read_excel('ENCODE_miRNA_experiments_02042016.xlsx', sheetname='Libraries')
rabi = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = rabi.parse('Libraries', header=0)

In [15]:
#library_alias_fixes = validate_aliases(libraries, 'library_accession')
#library_alias_fixes.to_csv('/tmp/library_fixes.csv')
#library_alias_fixes

In [16]:
created = []
if not libraries.empty:
    created = encode.post_sheet('/libraries/', libraries, dry_run=True, verbose=True)
    print(len(created))

0


In [17]:
if created:
    updates = []
    updatesdf = pandas.DataFrame(created, columns=['accession', 'uuid', 'aliases'])
    updatesdf.to_excel('/tmp/libraries.xlsx', index=False)

# Create Replicates

In [26]:
#replicates = pandas.read_excel('ENCODE_miRNA_experiments_02042016.xlsx', sheetname='Replicates')
rabi = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = rabi.parse('Replicates', header=0)

In [19]:
#replicates_alias_fixes = validate_aliases(replicates, 'uuid')
#replicates_alias_fixes.to_csv('/tmp/replicate_fixes.csv')
#replicates_alias_fixes

In [28]:
created = []
if not replicates.empty:
    created = encode.post_sheet('/replicates/', replicates, dry_run=True, verbose=True)
    print(len(created))

166


In [22]:
if created:
    updatesdf = pandas.DataFrame(created, columns=['uuid', 'experiment', 'library'])
    updatesdf.to_excel('/tmp/replicates.xlsx', index=False)

In [23]:
created

[{'aliases': ['ali-mortazavi:human-right cardiac atrium-ENTEX103-rep1'],
  'biological_replicate_number': 1,
  'experiment': 'ali-mortazavi:human-right cardiac atrium-ENTEX 103 104-nanostring',
  'library': 'ali-mortazavi:human-right cardiac atrium-ENTEX103-Lib1',
  'technical_replicate_number': 1,
  'uuid': 'would create'},
 {'aliases': ['ali-mortazavi:human-right cardiac atrium-ENTEX104-rep2'],
  'biological_replicate_number': 2,
  'experiment': 'ali-mortazavi:human-right cardiac atrium-ENTEX 103 104-nanostring',
  'library': 'ali-mortazavi:human-right cardiac atrium-ENTEX104-Lib2',
  'technical_replicate_number': 1,
  'uuid': 'would create'}]